<a href="https://colab.research.google.com/github/emiellemahieu/AOR/blob/master/AOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Code Master Dissertation:  'The Art of Roughness, a fractal and neural network combination approach to market risk measurement'

 This code uses standard parametric and non-parametric approaches to estimate Value-at-risk (VaR) and combines them in a neural net                                                   with the purpose to: 
 
 (1) Reduce the bias of the methods by combining them (Inui, Kijama, Itano, 2003)                                      
 (2) Explicitly tell the machine to learn a combination that minimizes the exceptions in financial loss (Kupiec, Christoffersen, 1998)                                          
 (3) Investigate the use of fractal complexity measures for this combination given the intimate relationship between fractional Brownian motion (fBM) (Mandelbrot,        Van Ness 1968 & Gatheral, 2014) and the generalization of standard assumptions in the classical models.                                            

SEE MAIN TEXT      [link text](https://)                                      


In [0]:
### CODE MASTER DISSERTATION:                                                                                                                        ###
###" The Art of Roughness:                                                                                                                           ###
###  A fractal and neural network combination approach to market risk measurement"                                                                   ###
###  2019               Emiel Lemahieu                                                                                                               ###
########################################################################################################################################################
########################################################################################################################################################
### This code uses classical parametric, non-parametric and Monte Carlo                                                                              ###
### approaches to estimate Value-at-risk (VaR) and combines them in a neural net                                                                     ###
### with the purpose to: (1) Reduce the bias of the methods by combining them (Inui, Kijama, Itano, 2003)                                            ###
###                      (2) Explicitly tell the machine to learn a combination                                                                      ###
###                          that minimizes the exceptions in financial loss (Kupiec, Christoffersen, 1998)                                          ###
###                      (3) Investigate the use of fractal complexity measures for this combination                                                 ###
###                          given the intimate relationship between fractional Brownian motion (fBM) (Mandelbrot, Van Ness 1968 & Gatheral, 2014)   ###
###                          and the generalization of standard assumptions in the classical models.                                                 ###
###                                                                                                                                                  ###
### SEE MAIN TEXT                                                                                                                                    ###
########################################################################################################################################################

### Required Installs

!pip install git+https://github.com/bashtage/arch.git #--install-option "--no-binary"
!pip install pandas_datareader
!pip install fbm
!pip install --upgrade setuptools
!pip install git+https://github.com/neuropsychology/NeuroKit.py.git #--install-option "--no-binary"
!pip install git+https://github.com/quandl/quandl-python.git
!pip install suds
!pip install pydatastream

################################################################################
### CURRENT COMMENTS / ROOM FOR IMPROVEMENT:                               #####
################################################################################

# (1) Multiprocessing for loops? 
# (2) What clusters to run it faster?
# (3) Still have to (properly) introduce Kupiec in loss function
# (4) Monte Carlo???


### df.fillna(df.mean(), inplace=True) !!!!!!!

# CHECK TICKERLIST in DO
# Genetics currently OFF to work on loss ftion
# Evaluate Vol options (GARCH/EGARCH/HARCH normal/t/skewt/ged)
# Plots for Vol are turned off !
# Shuffle the training set --> dubble check
# MCS??? -> TO BE DONE


################################################################################
### NECESSARY IMPORTS & PRESETS ################################################
################################################################################

from __future__ import absolute_import, division, print_function
import seaborn as sns
from scipy import stats
import tensorflow as tf
from tensorflow import keras
from tensorflow import keras
from keras import backend as K
import numpy as np
import pandas as pd 
import numpy as np
import scipy as sc
from numpy import cumsum, log, polyfit, sqrt, std, subtract
from numpy.random import randn
import math
import io
from scipy.stats import norm
from IPython import display
from google.colab import files
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset
from fbm import FBM
tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:f}'.format
import matplotlib as mpl
%matplotlib inline
from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
#mpl.rcParams['figure.dpi'] = 600
from google.colab import files
from datetime import datetime
from pandas_datareader import data, wb
from google.colab import files
import quandl
from pydatastream import Datastream
from functools import reduce
from operator import add
import random
from keras.datasets import mnist, cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from arch import arch_model
from mpl_toolkits.mplot3d import Axes3D
pd.options.mode.chained_assignment = None  # default='warn'
print(tf.__version__)
plt.ioff()

################################################################################
###        GENETIC ALGORITHMS          #########################################
################################################################################

"""
Class that holds a genetic algorithm for evolving a network.
Credit:
    A lot of this code was originally inspired by:
    http://lethain.com/genetic-algorithms-cool-name-damn-simple/
"""

# Helper: Early stopping.
early_stopper = EarlyStopping(patience=20)

def compile_model(network, train_data, test_data, train_labels, test_labels, dataframe,nh,n):
    """Compile a sequential model.
    Args:
        network (dict): the parameters of the network
    Returns:
        a compiled network.
    """
    # Get our network parameters.
    nb_layers = network['nb_layers']
    nb_neurons = network['nb_neurons']
    activation = network['activation']
    optimizer = network['optimizer']

    model = Sequential()

    # Add each layer.
    for i in range(nb_layers):

        # Need input shape for first layer.
        if i == 0:
            model.add(Dense(nb_neurons, activation=activation,input_shape=(train_data.shape[1],)))
        else:
            model.add(Dense(nb_neurons, activation=activation))

        model.add(Dropout(0.2))  # hard-coded dropout
    
    n=n
    df = dataframe
    nh = nh
    close='CLOSE'
    q=max(200,round(len(df[close])/5))
    n = len(df[close])-q
    n = len(df[close])-q
    
    def myloss(y_true,y_pred):
      
      
      customloss=keras.losses.mean_squared_error(y_true, y_pred)
      #INTRODUCE KUPIEC
      #int_y1 = K.cast([100*x for x in y_pred],'int32')
      
      returntensor = tf.convert_to_tensor(df['d_return'][nh:])
      #int_y2 = K.cast([100*x for x in df['d_return']],'int32')
      #int_y2 = K.cast([x for x in returntensor],'int32')
      
      y_pred = K.cast(y_pred, 'float32')
      returntensor = K.cast(returntensor, 'float32')
     
      errorGreater = K.cast(K.greater(returntensor,y_pred), 'float32')
      c=tf.reduce_sum(errorGreater)
      errorGreater2 = K.cast(K.greater(y_pred,-10000),'float32')
      total=tf.reduce_sum(errorGreater2)
      
      alpha = 1-cl
      p = c/total
      
      n0 = total-c
      n1 = c

      uc_h0 = n0 * np.log(1 - alpha) + n1 * np.log(alpha)
      uc_h1 = n0 * K.log(1 - p) + n1 * K.log(p)
      uc = -2 * (uc_h0 - uc_h1)
      #chi = tfp.distributions.Chi2(1, validate_args=False, allow_nan_stats=True, name='Chi2')
      chi = tf.distributions.Gamma(concentration=0.5 , rate=0.5, validate_args=False, allow_nan_stats=True, name='Gamma')
      #chi = tf.contrib.distributions.chi2(1)
      prob = 1-chi.cdf(uc, name= 'cdf')
      
      
      #customloss = p  #NO GRADIENTS AVAILABLE
      
      #customloss = keras.losses.mean_squared_error(y_true, y_pred) + p #'SOME' FAIL TO DIFFERENTIATE
      #k=5
      #customloss = k*(1-prob)+customloss
      #customloss = k*(1-prob)+customloss
      #customloss=10*(cl-p)*(cl-p)+customloss
      customloss=penalty*(cl-(1-p))*(cl-(1-p))+customloss
      return customloss
    
    
    model.compile(loss=myloss, optimizer=optimizer,
                  metrics=['accuracy','mae'])
    return model
  
def plot_history(history):
    plt.subplot(111)
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error')
    plt.plot(history.epoch, np.array(history.history['mean_absolute_error']),
             label='Train Loss')
    plt.plot(history.epoch, np.array(history.history['val_mean_absolute_error']),
             label = 'Val loss')
    plt.legend()
    plt.ylim([0, 0.0250])
    plt.show()
    
def get_data(train_data, test_data,train_labels,test_labels):
    """Retrieve the dataset and process the data."""
    # Set defaults.
    #nbclasses = 10
    batch_size = 64
    #input_shape = (3072,)

    # Get the data.
    x_train = train_data
    x_test = test_data
    y_train = train_labels
    y_test = test_labels
    
    #return (nb_classes, batch_size, input_shape, x_train, x_test, y_train, y_test)
    return (batch_size,x_train, x_test, y_train, y_test)

def train_and_score(network, dataset, train_data, test_data,train_labels,test_labels, dataframe,nh,n):
    """Train the model, return test loss.
    Args:
        network (dict): the parameters of the network
        dataset (str): Dataset to use for training/evaluating
    """
    if dataset == 'thesis':
        #nb_classes, batch_size, input_shape, x_train, \
        
            batch_size, x_train, x_test, y_train, y_test = get_data(train_data,test_data,train_labels,test_labels)
    #elif dataset == 'mnist':
     #   nb_classes, batch_size, input_shape, x_train, \
      #      x_test, y_train, y_test = get_mnist()

    model = compile_model(network, train_data, test_data, train_labels, test_labels, dataframe,nh,n)

    history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=1000,  # using early stopping, so no real limit
              verbose=0,
              validation_data=(x_test, y_test),
              callbacks=[early_stopper])
    
    train_predictions=model.predict(train_data)
    test_predictions=model.predict(test_data)

    score = model.evaluate(x_test, y_test, verbose=0)
    
    #return train_predictions, test_predictions, history, score[1]  # 1 is accuracy. 0 is loss.
    return train_predictions, test_predictions, history, score[0]  # 1 is accuracy. 0 is loss.
  
class Optimizer():
    """Class that implements genetic algorithm for MLP optimization."""

    def __init__(self, nn_param_choices, retain=0.4,
                 random_select=0.1, mutate_chance=0.2):
        """Create an optimizer.
        Args:
            nn_param_choices (dict): Possible network paremters
            retain (float): Percentage of population to retain after
                each generation
            random_select (float): Probability of a rejected network
                remaining in the population
            mutate_chance (float): Probability a network will be
                randomly mutated
        """
        self.mutate_chance = mutate_chance
        self.random_select = random_select
        self.retain = retain
        self.nn_param_choices = nn_param_choices

    def create_population(self, count):
        """Create a population of random networks.
        Args:
            count (int): Number of networks to generate, aka the
                size of the population
        Returns:
            (list): Population of network objects
        """
        pop = []
        for _ in range(0, count):
            # Create a random network.
            network = Network(self.nn_param_choices)
            network.create_random()

            # Add the network to our population.
            pop.append(network)

        return pop

    @staticmethod
    def fitness(network):
        """Return the accuracy, which is our fitness function."""
        #return network.accuracy
        return 1/network.accuracy

    def grade(self, pop):
        """Find average fitness for a population.
        Args:
            pop (list): The population of networks
        Returns:
            (float): The average accuracy of the population
        """
        summed = reduce(add, (self.fitness(network) for network in pop))
        return summed / float((len(pop)))

    def breed(self, mother, father):
        """Make two children as parts of their parents.
        Args:
            mother (dict): Network parameters
            father (dict): Network parameters
        Returns:
            (list): Two network objects
        """
        children = []
        for _ in range(2):

            child = {}

            # Loop through the parameters and pick params for the kid.
            for param in self.nn_param_choices:
                child[param] = random.choice(
                    [mother.network[param], father.network[param]]
                )

            # Now create a network object.
            network = Network(self.nn_param_choices)
            network.create_set(child)

            # Randomly mutate some of the children.
            if self.mutate_chance > random.random():
                network = self.mutate(network)

            children.append(network)

        return children

    def mutate(self, network):
        """Randomly mutate one part of the network.
        Args:
            network (dict): The network parameters to mutate
        Returns:
            (Network): A randomly mutated network object
        """
        # Choose a random key.
        mutation = random.choice(list(self.nn_param_choices.keys()))

        # Mutate one of the params.
        network.network[mutation] = random.choice(self.nn_param_choices[mutation])

        return network

    def evolve(self, pop):
        """Evolve a population of networks.
        Args:
            pop (list): A list of network parameters
        Returns:
            (list): The evolved population of networks
        """
        # Get scores for each network.
        graded = [(self.fitness(network), network) for network in pop]

        # Sort on the scores.
        graded = [x[1] for x in sorted(graded, key=lambda x: x[0], reverse=True)]

        # Get the number we want to keep for the next gen.
        retain_length = int(len(graded)*self.retain)

        # The parents are every network we want to keep.
        parents = graded[:retain_length]

        # For those we aren't keeping, randomly keep some anyway.
        for individual in graded[retain_length:]:
            if self.random_select > random.random():
                parents.append(individual)

        # Now find out how many spots we have left to fill.
        parents_length = len(parents)
        desired_length = len(pop) - parents_length
        children = []

        # Add children, which are bred from two remaining networks.
        while len(children) < desired_length:

            # Get a random mom and dad.
            male = random.randint(0, parents_length-1)
            female = random.randint(0, parents_length-1)

            # Assuming they aren't the same network...
            if male != female:
                male = parents[male]
                female = parents[female]

                # Breed them.
                babies = self.breed(male, female)

                # Add the children one at a time.
                for baby in babies:
                    # Don't grow larger than desired length.
                    if len(children) < desired_length:
                        children.append(baby)

        parents.extend(children)

        return parents
      
      
      
"""Class that represents the network to be evolved."""
import random
import logging


class Network():
    """Represent a network and let us operate on it.
    Currently only works for an MLP.
    """

    def __init__(self, nn_param_choices=None):
        """Initialize our network.
        Args:
            nn_param_choices (dict): Parameters for the network, includes:
                nb_neurons (list): [64, 128, 256]
                nb_layers (list): [1, 2, 3, 4]
                activation (list): ['relu', 'elu']
                optimizer (list): ['rmsprop', 'adam']
        """
        self.accuracy = 0.
        self.history={}
        self.test_predictions=None # Change back to [] 
        self.train_predictions=None
        self.nn_param_choices = nn_param_choices
        self.network = {}  # (dic): represents MLP network parameters

    def create_random(self):
        """Create a random network."""
        for key in self.nn_param_choices:
            self.network[key] = random.choice(self.nn_param_choices[key])

    def create_set(self, network):
        """Set network properties.
        Args:
            network (dict): The network parameters
        """
        self.network = network

    def train(self, dataset, train_data,test_data,train_labels,test_labels,dataframe,nh,n):
        """Train the network and record the accuracy.
        Args:
            dataset (str): Name of dataset to use.
        """
        if self.accuracy == 0.:
            self.train_predictions, self.test_predictions, self.history, self.accuracy = train_and_score(self.network, dataset, train_data,test_data,train_labels,test_labels,dataframe,nh,n)

    def print_network(self):
        """Print out a network."""
        print(self.network)
        #print("Network accuracy: %.2f%%" % (self.accuracy * 100))
        print("Network score: %.2f%" % (self.accuracy * 100))
        #plot_history(self.history)
    
    def network_test_predictions(self):
      return self.test_predictions
    
    def network_train_predictions(self):
      return self.train_predictions
        
import logging
#from optimizer import Optimizer
from tqdm import tqdm

# Setup logging.
logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%m/%d/%Y %I:%M:%S %p',
    level=logging.DEBUG,
    filename='log.txt'
)

def train_networks(networks, dataset, train_data,test_data,train_labels,test_labels,dataframe,nh,n):
    """Train each network.
    Args:
        networks (list): Current population of networks
        dataset (str): Dataset to use for training/evaluating
    """
    pbar = tqdm(total=len(networks))
    for network in networks:
        network.train(dataset, train_data,test_data,train_labels,test_labels,dataframe,nh,n)
        pbar.update(1)
    pbar.close()

def get_average_accuracy(networks):
    """Get the average accuracy for a group of networks.
    Args:
        networks (list): List of networks
    Returns:
        float: The average accuracy of a population of networks.
    """
    total_accuracy = 0
    for network in networks:
        total_accuracy += network.accuracy

    return total_accuracy / len(networks)

def generate(generations, population, nn_param_choices, dataset, train_data,test_data,train_labels,test_labels,dataframe,nh,n):
    """Generate a network with the genetic algorithm.
    Args:
        generations (int): Number of times to evole the population
        population (int): Number of networks in each generation
        nn_param_choices (dict): Parameter choices for networks
        dataset (str): Dataset to use for training/evaluating
    """
    optimizer = Optimizer(nn_param_choices)
    networks = optimizer.create_population(population)

    # Evolve the generation.
    for i in range(generations):
        print("***Doing generation %d of %d***" %
                     (i + 1, generations))

        # Train and get accuracy for networks.
        train_networks(networks, dataset, train_data,test_data,train_labels,test_labels,dataframe,nh,n)

        # Get the average accuracy for this generation.
        average_accuracy = get_average_accuracy(networks)

        # Print out the average accuracy each generation.
        #print('-'*80)
        #print("Generation average: %.2f%%" % (average_accuracy * 100))
        #print('-'*80)

        # Evolve, except on the last iteration.
        if i != generations - 1:
            # Do the evolution.
            networks = optimizer.evolve(networks)

    # Sort our final population.
    networks = sorted(networks, key=lambda x: x.accuracy, reverse=True)
    
    return networks[:1]

def print_networks(networks):
    """Print a list of networks.
    Args:
        networks (list): The population of networks
    """
    print('-'*80)
    for network in networks:
        network.print_network()
        
def get_network_train_predictions(networks):
  
  for network in networks: 
    its=network.network_train_predictions()
  return its

def get_network_test_predictions(networks):
  
  for network in networks: 
    its=network.network_test_predictions()
  return its


################################################################################
##### CODE FOR MODEL BACKTESTING AND BASIS FOR CUSTOM LOSS FTN #################
################################################################################
### The main code from this class is based on the Kupiec-Christoffersen 1998   #########
### framework which uses a loglikelihood ratio to evaluate the number of exceptions  ###
### in financial loss.                                                               ###
########################################################################################

class Backtest:
    def __init__(self, actual, forecast, alpha):
        self.index = actual.index
        self.actual = actual.values
        self.forecast = forecast.values
        self.alpha = alpha

    def hit_series(self):
        return (self.actual < self.forecast) * 1

    def number_of_hits(self):
        return self.hit_series().sum()

    def hit_rate(self):
        return self.hit_series().mean()

    def expected_hits(self):
        return self.actual.size * self.alpha

    def duration_series(self):
        hit_series = self.hit_series()
        hit_series[0] = 1
        hit_series[-1] = 1
        return np.diff(np.where(hit_series == 1))[0]

    def plot(self, file_name=None):

        # Re-add the time series index
        r = pd.Series(self.actual, index=self.index)
        q = pd.Series(self.forecast, index=self.index)

        sns.set_context("paper")
        sns.set_style("whitegrid", {"font.family": "serif", "font.serif": "Computer Modern Roman", "text.usetex": True})

        # Hits
        ax = r[r <= q].plot(color="red", marker="o", ls="None", figsize=(6, 3.5))
        for h in r[r <= q].index:
            plt.axvline(h, color="black", alpha=0.4, linewidth=1, zorder=0)

        # Positive returns
        r[q < r].plot(ax=ax, color="green", marker="o", ls="None")

        # Negative returns but no hit
        r[(q <= r) & (r <= 0)].plot(ax=ax, color="orange", marker="o", ls="None")

        # VaR
        q.plot(ax=ax, grid=False, color="black", rot=0)

        # Axes
        plt.xlabel("")
        plt.ylabel("Log Return")
        ax.yaxis.grid()

        sns.despine()
        if file_name is None:
            plt.show()
        else:
            plt.savefig(file_name, bbox_inches="tight")
        plt.close("all")

    def tick_loss(self, return_mean=True):
        loss = (self.alpha - self.hit_series()) * (self.actual - self.forecast)
        if return_mean:
            return loss.mean()
        else:
            return loss

    def smooth_loss(self, delta=25, return_mean=True):
        """Gonzalez-Rivera, Lee and Mishra (2004)"""
        loss = ((self.alpha - (1 + np.exp(delta*(self.actual - self.forecast)))**-1) * (self.actual - self.forecast))
        if return_mean:
            return loss.mean()
        else:
            return loss

    def quadratic_loss(self, return_mean=True):
        """Lopez (1999); Martens et al. (2009)"""
        loss = (self.hit_series() * (1 + (self.actual - self.forecast)**2))
        if return_mean:
            return loss.mean()
        else:
            return loss

    def firm_loss(self, c=1, return_mean=True):
        """Sarma et al. (2003)"""
        loss = (self.hit_series() * (1 + (self.actual - self.forecast)**2) - c*(1-self.hit_series()) * self.forecast)
        if return_mean:
            return loss.mean()
        else:
            return loss

    def lr_bt(self):
        """Likelihood ratio framework of Christoffersen (1998)"""
        hits = self.hit_series()   # Hit series
        tr = hits[1:] - hits[:-1]  # Sequence to find transitions

        # Transitions: nij denotes state i is followed by state j nij times
        n01, n10 = (tr == 1).sum(), (tr == -1).sum()
        n11, n00 = (hits[1:][tr == 0] == 1).sum(), (hits[1:][tr == 0] == 0).sum()

        # Times in the states
        n0, n1 = n01 + n00, n10 + n11
        n = n0 + n1
        print('no '+str(n0))
        print('n1 '+str(n1))
        print('n '+str(n))

        # Probabilities of the transitions from one state to another
        p01, p11 = n01 / (n00 + n01), n11 / (n11 + n10)
        p = n1 / n
        print(str(p))

        if n1 > 0:
            # Unconditional Coverage
            uc_h0 = n0 * np.log(1 - self.alpha) + n1 * np.log(self.alpha)
            uc_h1 = n0 * np.log(1 - p) + n1 * np.log(p)
            uc = -2 * (uc_h0 - uc_h1)

            # Independence
            ind_h0 = (n00 + n01) * np.log(1 - p) + (n01 + n11) * np.log(p)
            ind_h1 = n00 * np.log(1 - p01) + n01 * np.log(p01) + n10 * np.log(1 - p11)
            if p11 > 0:
                ind_h1 += n11 * np.log(p11)
            ind = -2 * (ind_h0 - ind_h1)

            # Conditional coverage
            cc = uc + ind

            # Stack results
            df = pd.concat([pd.Series([uc, ind, cc]),
                            pd.Series([1 - stats.chi2.cdf(uc, 1),
                                       1 - stats.chi2.cdf(ind, 1),
                                       1 - stats.chi2.cdf(cc, 2)])], axis=1)
        else:
            df = pd.DataFrame(np.zeros((3, 2))).replace(0, np.nan)

        # Assign names
        df.columns = ["Statistic", "p-value"]
        df.index = ["Unconditional", "Independence", "Conditional"]

        return df,p

    def dq_bt(self, hit_lags=4, forecast_lags=1):
        """Dynamic Quantile Test (Engle & Manganelli, 2004)"""
        try:
            hits = self.hit_series()
            p, q, n = hit_lags, forecast_lags, hits.size
            pq = max(p, q - 1)
            y = hits[pq:] - self.alpha  # Dependent variable
            x = np.zeros((n - pq, 1 + p + q))
            x[:, 0] = 1  # Constant

            for i in range(p):  # Lagged hits
                x[:, 1 + i] = hits[pq-(i+1):-(i+1)]

            for j in range(q):  # Actual + lagged VaR forecast
                if j > 0:
                    x[:, 1 + p + j] = self.forecast[pq-j:-j]
                else:
                    x[:, 1 + p + j] = self.forecast[pq:]

            beta = np.dot(np.linalg.inv(np.dot(x.T, x)), np.dot(x.T, y))
            lr_dq = np.dot(beta, np.dot(np.dot(x.T, x), beta)) / (self.alpha * (1-self.alpha))
            p_dq = 1 - stats.chi2.cdf(lr_dq, 1+p+q)

        except:
            lr_dq, p_dq = np.nan, np.nan

        return pd.Series([lr_dq, p_dq],
                         index=["Statistic", "p-value"], name="DQ")
      
#########################################################################################
##### COMPLEXITY MEASURES: ALGORITHMS                          ##########################
#########################################################################################
### The basic reasoning and set-up of the complexity measures and algorithms        ###
### are discussed in the main text.                                                   ###
### In summary, fractional dimensions (Katz, Higushi) measure the complexity that     ###
### is implied in the stock price by calculting how the power of the time step scales ###
### with the 'covered distance'. Hurst exponents measure how the rescaled range of    ###
### smaller time step intervals scales with the time step size.                       ### 
#########################################################################################

def katz(data):
    n = len(data)-1
    L = np.hypot(np.diff(data), 1).sum() # Sum of distances
    d = np.hypot(data - data[1], np.arange(len(data))).max() # furthest distance from first point
    return log(n) / (log( d/L ) + log( n))

def katz2(data):
  
    first=data.index[0]
    dists = np.hypot(np.diff(data), 1)
    L = dists.sum()
    a = dists.mean()
    d = np.hypot(data - first, np.arange(len(data))).max() # furthest distance from first point
    return np.log( L/a ) / np.log( d/a)

def complexity_fd_higushi(signal, k_max):
    """
    Computes Higuchi Fractal Dimension of a signal. Based on the `pyrem <https://github.com/gilestrolab/pyrem>`_ repo by Quentin Geissmann.
    Parameters
    ----------
    signal : list or array
        List or array of values.
    k_max : int
        The maximal value of k. The point at which the FD plateaus is considered a saturation point and that kmax value should be selected (Gómez, 2009). Some studies use a value of 8 or 16 for ECG signal and other 48 for MEG.
    Returns
    ----------
    fd_higushi : float
        The Higushi Fractal Dimension as float value.
    Example
    ----------
    >>> import neurokit as nk
    >>>
    >>> signal = np.sin(np.log(np.random.sample(666)))
    >>> fd_higushi = nk.complexity_fd_higushi(signal, 8)
    Notes
    ----------
    *Details*
    - **Higushi Fractal Dimension**: Higuchi proposed in 1988 an efficient algorithm for measuring the FD of discrete time sequences. As the reconstruction of the attractor phase space is not necessary, this algorithm is simpler and faster than D2 and other classical measures derived from chaos theory. FD can be used to quantify the complexity and self-similarity of a signal. HFD has already been used to analyse the complexity of brain recordings and other biological signals.
    *Authors*
    - Quentin Geissmann (https://github.com/qgeissmann)
    *Dependencies*
    - numpy
    *See Also*
    - pyrem package: https://github.com/gilestrolab/pyrem
    References
    -----------
    - Accardo, A., Affinito, M., Carrozzi, M., & Bouquet, F. (1997). Use of the fractal dimension for the analysis of electroencephalographic time series. Biological cybernetics, 77(5), 339-350.
    - Gómez, C., Mediavilla, Á., Hornero, R., Abásolo, D., & Fernández, A. (2009). Use of the Higuchi's fractal dimension for the analysis of MEG recordings from Alzheimer's disease patients. Medical engineering & physics, 31(3), 306-313.
    """
    signal = np.array(signal)
    L = []
    x = []
    N = signal.size

    km_idxs = np.triu_indices(k_max - 1)
    km_idxs = k_max - np.flipud(np.column_stack(km_idxs)) -1
    km_idxs[:,1] -= 1


    for k in range(1, k_max):
        Lk = 0
        for m in range(0, k):
            #we pregenerate all idxs
            idxs = np.arange(1,int(np.floor((N-m)/k)))

            Lmk = np.sum(np.abs(signal[m+idxs*k] - signal[m+k*(idxs-1)]))
            Lmk = (Lmk*(N - 1)/(((N - m)/ k)* k)) / k
            Lk += Lmk

        if Lk != 0:
            L.append(np.log(Lk/(m+1)))
            x.append([np.log(1.0/ k), 1])

    (p, r1, r2, s)=np.linalg.lstsq(x, L)
    fd_higushi = p[0]
    return (fd_higushi)
def complexity_entropy_shannon(signal):
    """
    Computes the shannon entropy. Copied from the `pyEntropy <https://github.com/nikdon/pyEntropy>`_ repo by tjugo.
    Parameters
    ----------
    signal : list or array
        List or array of values.
    Returns
    ----------
    shannon_entropy : float
        The Shannon Entropy as float value.
    Example
    ----------
    >>> import neurokit as nk
    >>>
    >>> signal = np.sin(np.log(np.random.sample(666)))
    >>> shannon_entropy = nk.complexity_entropy_shannon(signal)
    Notes
    ----------
    *Details*
    - **shannon entropy**: Entropy is a measure of unpredictability of the state, or equivalently, of its average information content.
    *Authors*
    - tjugo (https://github.com/nikdon)
    *Dependencies*
    - numpy
    *See Also*
    - pyEntropy package: https://github.com/nikdon/pyEntropy
    References
    -----------
    - None
    """
     # Check if string
    if not isinstance(signal, str):
        signal = list(signal)

    signal = np.array(signal)


    # Create a frequency data
    data_set = list(set(signal))
    freq_list = []
    for entry in data_set:
        counter = 0.
        for i in signal:
            if i == entry:
                counter += 1
        freq_list.append(float(counter) / len(signal))

    # Shannon entropy
    shannon_entropy = 0.0
    for freq in freq_list:
        shannon_entropy += freq * np.log2(freq)
    shannon_entropy = -shannon_entropy

    return(shannon_entropy)
  
def bootstrap_resample(X, n=None):
    """ Bootstrap resample an array_like
    Parameters
    ----------
    X : array_like
      data to resample
    n : int, optional
      length of resampled array, equal to len(X) if n==None
    Results
    -------
    returns X_resamples
   
    if n == None:
        n = len(X)
        
    resample_i = np.floor(np.random.rand(n)*len(X)).astype(int)
    X_resample = X[resample_i]
    """
    X_resample = np.random.choice(X, size=n, replace=True)
    return X_resample

def hurst(ts):
  #Returns the Hurst Exponent of the time series vector ts
  # Create the range of lag values
  lags = range(2, 100)

  # Calculate the array of the variances of the lagged differences
  tau = [sqrt(std(subtract(ts[lag:], ts[:-lag]))) for lag in lags]
  
  # Use a linear fit to estimate the Hurst Exponent
  poly = polyfit(log(lags), log(tau), 1)
  #plt.scatter(log(lags),log(tau)) 
  
  # Return the Hurst exponent from the polyfit output
  return (poly[0]*2.0)

################################################################################
#### VOLATILITY ESTIMATION            ##########################################
################################################################################

def vol_est(series, vol_type,n,ticker,error):

  returns =((series[close]-series[close].shift(1)) / series[close].shift(1) )
  
  #OPTIONS: GARCH/EGARCH/ARCH/HARCH normal/t/skewt/ged
  g = arch_model(returns, vol=vol_type,p=1,o=0,q=1,dist=error,hold_back=1)
  res = g.fit(update_freq=0, disp='off')
  #print(res.summary())
  ##fig = res.plot(annualize='D')
  sigma=res.conditional_volatility
  #plt.subplot(211)
  #plt.plot(sigma, label = type+" "+ticker, color='r')
  #_=plt.legend()
  #plt.subplot(212)
  #plt.plot(returns, label= ticker+" daily return",color='g')
  #_ = plt.legend()
  #plt.show()
  return sigma

################################################################################
### fBM SIMULATION                     #########################################
################################################################################


def get_fBM_SIM(h,mean,vol):
## Still bullshit: check !!!!
  hu=0
  if(0>=h):
    hu = 0.1
  else:
    if(h>=1):
      hu = 0.8
    else: 
      hu = h
    
  f = FBM(n=1000, hurst=hu, length=1, method='daviesharte')
  # or
  #f = FBM(100, 0.5)
  # Generate a fBm realization
  #fbm_sample = f.fbm()*1000
  # Generate a fGn realization
  fgn_sample = f.fgn()

  # Get the times associated with the fBm
  t_values = f.times()

  hVaR = -np.min(fgn_sample)
  return hVaR

################################################################################
###  DEFINITIONS LOOKBACK WINDOW N OBS #########################################
################################################################################

def Mean_n_close(x,n,df):
#Mean of last n closing prices
  loc = df[close][df[close]==x].index[0]
  M = np.mean(df.loc[(loc-n):loc, close])
  
  return M

def hs_n_return(x,n, confidence, df):
#Empirical quantiles last n returns
  loc = df['d_return'][df['d_return']==x].index[0]
  ret = -df.loc[(loc-n):loc, 'd_return']
  hs = np.percentile(ret, confidence)
  
  return hs

def simple_fhs_n_return(x,n, confidence, df):
#Empirical quantiles last n returns filtered for GARCH vol
  loc = df['d_return'][df['d_return']==x].index[0]
  vol = df.loc[(loc-n):loc, 'GARCH_vol']
  ret = df.loc[(loc-n):loc, 'd_return'] 
  idd = ret/vol
  fhs = np.percentile(idd, confidence)
  
  return fhs

def Mean_n_return(x,n,df):
#Mean of last n returns
  l = df['d_return'][df['d_return']==x]
  loc = df['d_return'][df['d_return']==x].index[0]
  M = np.mean(df.loc[(loc-n):loc, 'd_return'])
  
  return M

def Kurt_n_return(x,n,df):
#Kurtosis of last n returns
  l = df['d_return'][df['d_return']==x]
  loc = df['d_return'][df['d_return']==x].index[0]
  K = sc.stats.kurtosis(df.loc[(loc-n):loc, 'd_return'])
  
  return K

def Hurst_n_close(x,n,df):
#Kurtosis of last n returns
  l = df[close][df[close]==x]
  loc = df[close][df[close]==x].index[0]
  rough_series = df.loc[(loc-n):loc, close]
  #geared_rough_series = bootstrap_resample(rough_series,1000) -> don't do this
  #h = hurst(geared_rough_series)
  h=hurst(rough_series)
  return h

def true_n_VaR(x,n,cl,df):
#Empirical quantiles for 'next' n returns -> TO BE ABANDONNED!!!!
  l = df['d_return'][df['d_return']==x]
  loc = df['d_return'][df['d_return']==x].index[0]
  rough_series = df.loc[loc:(loc+n), 'd_return']
  c=np.percentile(rough_series,cl)
  return -c
  

def higushi_n_close(x,n,df):
#Higushi FB of last n closing prices chart
  l = df[close][df[close]==x]
  loc = df[close][df[close]==x].index[0]
  rough_series = df.loc[(loc-n):loc, close]
  k = 16
  c=complexity_fd_higushi(rough_series,k)
  return c

def shannon_n_close(x,n,df):
#Shannon entropy of last n closing prices chart
  l = df[close][df[close]==x]
  loc = df[close][df[close]==x].index[0]
  rough_series = df.loc[(loc-n):loc, close]
  c=complexity_entropy_shannon(rough_series)
  return c

def katz3(x,n,df):
#Katz FD of last n closing prices chart
  l = df[close][df[close]==x]
  loc = df[close][df[close]==x].index[0]
  rough_series = df.loc[(loc-n):loc, close]
  h=katz2(rough_series)
  return h

################################################################################
### PRESETS DATA SOURCE                   ######################################
################################################################################

#Pick a data source: datastream (Preferred),'yahoo' (BUG) 'google' (BUG) 'quandl' (COMPLETENESS?) 'morningstar' (BUG) 'iex' (INVESTOR EXCHANGE: ONLY AMERICAN)

res = pd.DataFrame(data=None, dtype=np.int8)
source = "datastream" #@param ["iex", "yahoo", "google", "morningstar", "quandl", "datastream"]
close=None

if(source == 'morningstar' or source =='google' or source =='quandl'):
      close='Close'
      volume='Volume'
else:
      if(source=="iex"):
        close = 'close'
        volume='volume'
      else: close = "Adj Close"
if(source=='datastream'):
    close='CLOSE'
    volume='VOL'

    
################################################################################
### MAIN FUNCTION: ANALYSIS IN 'DO'                    #########################
################################################################################

def do(t,count, nr_of_generations, population_size):  
  
# Trains a neural net based on different VaRs and fractal complexity measures of
# stock with ticker t, over 'nr_of_generations' generations with an initial population
# size of 'population_size'. Returns a result dataframe res that is subsequently absorbed
# by a summary dataframe summ that is filled with the different stocks over time and their backtest results.
  
  ticker = t 

  starting_date = "2004-03-01" #@param {type:"date"}
  ending_date = "2019-03-01" #@param {type:"date"}
  
  if(source=='datastream'):
    df=DWE.get_OHLCV(ticker,date_from=starting_date,date_to=ending_date)
    #df = DWE.fetch(ticker, ['PI','VO'], date_from=starting_date,date_to=ending_date)
    df = df.reset_index(drop=True)
  lookback_period = 500 #@param {type:"integer"}
  confidence_level = 0.99 #@param {type:"number"}
  hurst_n = 500 #@param {type:"number"}
  nh = hurst_n
  lb = lookback_period
  cl = confidence_level
 
  """
  df2=pd.DataFrame(data=None)
  df2['CLOSE']=df['PI']
  df2['VOL']=df['VO']
  df=df2
  """
  
  print(ticker)
  print(source)
  print(starting_date)
  print(ending_date)
  
  data_source=source
  start_date = starting_date
  end_date = ending_date
  # Download

  if(source!='datastream'):
    df = data.DataReader(ticker , data_source, start_date, end_date)
    df = df.reset_index(drop=True)
    
  #df.dropna(inplace=True)
  df.fillna(df.mean(), inplace=True)
  
  #Calculate daily logreturns
  df['d_return'] = log(df[close] / df[close].shift(1))

  plt.plot(df[close], label = ticker+" quote "+starting_date+" - "+ending_date)
  _=plt.legend()
  plt.show()
  
  ### Estimation of daily GARCH volatility with following options:
  ###    * GARCH/EGARCH/ARCH/HARCH  with normal/t/skewt/ged errors -->  !!! EVALUATE !!! 
  
  df['GARCH_vol'] = vol_est(df, 'GARCH',412,ticker,'normal')
  #df['EGARCH_vol'] = vol_est(df, 'EGARCH',413,ticker,'normal')
  #df['HARCH_vol'] = vol_est(df, 'HARCH',414, ticker,'normal')
  #df['GARCH_vol'] = vol_est(df, 'GARCH',412,ticker,'t')
  #df['ARCH_vol'] = vol_est(df, 'EGARCH',413,ticker,'t')
  #df['EGARCH_vol'] = vol_est(df, 'ARCH',413,ticker,'t')
  #df['HARCH_vol'] = vol_est(df, 'HARCH',414, ticker,'t')
  #df['GARCH_vol'] = vol_est(df, 'GARCH',412,ticker,'skewt')
  #df['ARCH_vol'] = vol_est(df, 'EGARCH',413,ticker,'skewt')
  #df['EGARCH_vol'] = vol_est(df, 'HGARCH',413,ticker,'skewt')
  #df['HARCH_vol'] = vol_est(df, 'ARCH',414, ticker,'skewt')
  #df['GARCH_vol'] = vol_est(df, 'GARCH',412,ticker,'ged')
  #df['ARCH_vol'] = vol_est(df, 'EGARCH',413,ticker,'ged')
  #df['EGARCH_vol'] = vol_est(df, 'HGARCH',413,ticker,'ged')
  #df['HARCH_vol'] = vol_est(df, 'ARCH',414, ticker,'ged')

  df["Mean_"+str(lb)] = 0.00
  for i in range(1,len(df['d_return'])):
    df["Mean_"+str(lb)][i]=Mean_n_return(df['d_return'][i],lb,df)

  df["Empirical_VaR_"+str(lb)] = 0.00
  for i in range(1,len(df['d_return'])):
    df["Empirical_VaR_"+str(lb)][i]=true_n_VaR(df['d_return'][i],lb/2,(1-cl),df)


  df["Hurst_"+str(nh)] = 0.00
  for i in range((nh-1),len(df[close])):
    df["Hurst_"+str(nh)][i]=Hurst_n_close(df[close][i],nh,df)

  #df['fBM_SIM']= 0
  #for i in range(0,len(df[close])):
  #  df['fBM_SIM'][i] = get_fBM_SIM(df["Hurst_"+str(nh)][i], df["Mean_"+str(lb)][i],df['GARCH_vol'][i])
  
  df["Higushi_FD_"+str(nh)]=1.5
  for i in range((nh-1), len(df[close])):
    df['Higushi_FD_'+str(nh)][i]= higushi_n_close(df[close][i],nh,df)
  
  #df["Shannon_"+str(nh)]=5
  #for i in range((nh-1), len(df[close])):
  #  df['Shannon_'+str(nh)][i]= shannon_n_close(df[close][i],nh,df)

  df["Katz_"+str(nh)]=1.5
  for i in range((nh-1), len(df[close])):
    df['Katz_'+str(nh)][i]= katz3(df[close][i],nh,df)

  df['NVaR']=  - df["Mean_"+str(lb)] + df['GARCH_vol']*(norm.ppf(cl))
  
  #NES(n,1) = port*(-mean(returns)+(normpdf(norminv(cl))/(1-cl))*std(returns))
  #v = (6+4*Kurt_n_return(df['d_return'],lb))/(Kurt_n_return(df['d_return'],lb)) 
  #TVAR(n,1) = port*(-mean(returns) + tinv(cl,v) * sqrt((v-2)/v) * std(returns) )
  #LVAR(n,1) = port* ( 1-exp( mean(returns) - std(returns)*norminv(cl) ) )

  df['NES']= - df["Mean_"+str(lb)] + df['GARCH_vol']*(norm.pdf((norm.ppf(cl)))/(1-cl))

  df['TVaR']=0
  for i in range(1,len(df[close])):
    v = (6+4*Kurt_n_return(df['d_return'][i],nh,df))/(Kurt_n_return(df['d_return'][i],nh,df))
    df['TVaR'][i]= - df["Mean_"+str(lb)][i] + sc.stats.t.ppf(cl,v) * np.sqrt((v-2)/v) * df['GARCH_vol'][i]

  df['logVaR']= (1-np.exp(df["Mean_"+str(lb)] - df['GARCH_vol']*(norm.ppf(cl))))
  
  df['HS']=0
  for i in range(1,len(df[close])):
    df['HS'][i]= hs_n_return(df['d_return'][i],nh,cl,df)
  df['HS']=-df['HS']
  
  df['FHS']=0
  for i in range(1,len(df[close])):
    df['FHS'][i]= simple_fhs_n_return(df['d_return'][i],nh,cl,df)*df['GARCH_vol'][i]
  df['FHS']=-df['FHS']
  
  eta = 0.2
  df['FVaR']=  df["Mean_"+str(lb)] - ( df['GARCH_vol'] / eta )*(1-np.power(-np.log(cl),-eta))
  df['GVaR']=  df["Mean_"+str(lb)] - df['GARCH_vol']*np.log(-np.log(cl))
  
  df.fillna(df.mean(), inplace=True)
  
  display.display(df.describe())
  display.display(df.head())
  display.display(df.tail())


  o=np.mean(df['Hurst_'+str(nh)][nh:])

  if(count==0):
    plt.subplot(111)
    plt.plot(df['d_return'][nh:], label='return')
    _=plt.legend()
    plt.show()
    plt.subplot(111)
    plt.plot(df["Hurst_"+str(nh)][nh:],label='Hurst',color='y',marker='|')
    k=pd.Series(len(df['Hurst_'+str(nh)]))
    for i in range(0,len(df['Hurst_'+str(nh)])):
      k[i]=o
    plt.plot(k[nh:], label='Mean Hurst')
    _=plt.legend()
    plt.show()
    #plt.subplot(111)
    #plt.plot(df['Shannon_'+str(nh)],label=ticker)
    #_=plt.legend()
    #plt.show()
    plt.subplot(111)
    plt.plot(df[close][nh:],label=ticker)
    _=plt.legend()
    plt.show()
    plt.subplot(111)
    plt.plot(df['GARCH_vol'][nh:], label='GARCH Vol', color='r')
    _=plt.legend()
    plt.subplot(111)
    plt.plot(df['NVaR'][nh:], label = 'NVaR')
    plt.plot(df['TVaR'][nh:],label="TVaR", c='c')
    plt.plot(df['FVaR'][nh:],label="FVaR", c='b')
    _=plt.legend()
    plt.show()
    plt.subplot(111)
    plt.plot(df['GVaR'][nh:],label="GVaR", c='g')
    plt.plot(df['FHS'][nh:],label="FHS", c='r')
    plt.plot(df['HS'][nh:],label="HS", c='y')
    _=plt.legend()
    plt.show()
  #fig = plt.figure()
  #ax1 = fig.add_subplot(111, projection='3d')
  #Axes3D.plot_trisurf(ax1,df['Hurst_'+str(nh)][nh:], df['GARCH_vol'][nh:],df['NVaR'][nh:])
  #Axes3D.scatter(ax1,df['Hurst_'+str(nh)][nh:],df['GARCH_vol'][nh:],df['logVaR'][nh:],cmap='seismic', c=df['logVaR'][nh:].shift(-1) / np.max(df['NVaR'][nh:]))

  
  ##############################################################################
  ### START OF MACHINE LEARNING: PREPROCESSING                 #################
  ##############################################################################
  
  def preprocess_features(df):
    selected_features =df[
      [
       volume,
       "HS",
       "FHS",
       "FVaR",
       "GVaR",
       #"Katz_"+str(nh),
       "Hurst_"+str(nh),
       "GARCH_vol",
       "Higushi_FD_"+str(nh),
       #"Shannon_"+str(nh),
       "NVaR",
       "TVaR",
       "logVaR" 
      ]]

    processed_features = selected_features.copy()
    # Create a synthetic feature?

    processed_features[volume] = df[volume]/(df[volume].max())


    #processed_features["Katz_"+str(nh)] = df["Katz_"+str(nh)]/(2)
    #processed_features["Higushi_FD_"+str(nh)] = df["Higushi_FD_"+str(nh)]/(2)
    #processed_features["Shannon_"+str(nh)] = df["Shannon_"+str(nh)]/(df["Shannon_"+str(nh)].max())
    return processed_features
    
  def preprocess_features_wo_aor(df):
    selected_features =df[
      [
       "HS",
       "FHS",
       "FVaR",
       "GVaR",
       "NVaR",
       "TVaR",
       "logVaR" 
      ]]

    processed_features = selected_features.copy()
    # Create a synthetic feature?

    #processed_features[volume] = df[volume]/(df[volume].max())


    #processed_features["Katz_"+str(nh)] = df["Katz_"+str(nh)]/(2)
    #processed_features["Higushi_FD_"+str(nh)] = df["Higushi_FD_"+str(nh)]/(2)
    #processed_features["Shannon_"+str(nh)] = df["Shannon_"+str(nh)]/(df["Shannon_"+str(nh)].max())


    return processed_features
  

  def preprocess_targets(df):

    output_targets = pd.DataFrame()
    output_targets["Empirical_VaR_"+str(lb)] = df['Empirical_VaR_'+str(lb)]
    
    return output_targets

  # Choose the first examples for training.

  ### !!!! Reconsider this test vs validation sample distinction !!!!
  
  q=max(200,round(len(df[close])/5))
  n = len(df[close])-q
  n = len(df[close])-q
  training_examples = preprocess_features(df[nh:n])
  training_targets = preprocess_targets(df[nh:n])
  train_data_wo_aor = preprocess_features_wo_aor(df[nh:n])

  # Choose the last q examples for validation.
  validation_examples = preprocess_features(df.tail(q))
  validation_targets = preprocess_targets(df.tail(q))
  test_data_wo_aor = preprocess_features_wo_aor(df.tail(q))
  
  validation_returns = df["d_return"].tail(q)
  test_nvar = df['NVaR'].tail(q)
  test_tvar = df['TVaR'].tail(q)
  test_logvar = df['logVaR'].tail(q)
  test_fvar = df['FVaR'].tail(q)
  test_FHS = df['FHS'].tail(q)
  test_gvar = df['GVaR'].tail(q)
  test_HS = df['HS'].tail(q)
  """
  # Double-check that we've done the right thing.
  if(count==0):
    print("Training examples summary:")
    display.display(training_examples.describe())
    print("Validation examples summary:")
    display.display(validation_examples.describe())

    print("Training targets summary:")
    display.display(training_targets.describe())
    print("Validation targets summary:")
    display.display(validation_targets.describe())
    
    plt.subplot(111)
    plt.plot(validation_targets, label= ticker+" NY plot",color='g')
    _ = plt.legend()
    plt.show()
    """ 
  
  def construct_feature_columns(input_features):
    return set([tf.feature_column.numeric_column(my_feature)
                for my_feature in input_features])
  
  def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):

    features = {key:np.array(value) for key,value in dict(features).items()}                                           

   # Construct a dataset, and configure batching/repeating.
    ds = Dataset.from_tensor_slices((features,targets)) # warning: 2GB limit
    ds = ds.batch(batch_size).repeat(num_epochs)
  # Shuffle the data, if specified.
    if shuffle:
      ds = ds.shuffle(1000)

  # Return the next batch of data.
    features, labels = ds.make_one_shot_iterator().get_next()
    return features, labels
  
  train_data=training_examples 
  train_labels=training_targets
  test_data=validation_examples
  test_labels=validation_targets

  # Shuffle the training set  --> dubble check this
  # order = np.argsort(np.random.random(train_labels.shape))
  # train_data = train_data[order]
  # train_labels = train_labels[order]

  print("Training set: {}".format(train_data.shape)) 
  print("Testing set:  {}".format(test_data.shape)) 

  print(train_labels[0:10])
  
  ##############################################################################
  ### LET'S BUILD OUR NEURAL NETWORK                           #################
  ##############################################################################
  
  
  
  #### Comment/Uncomment next section not to/to have genetic algorithms: meant for development purposes
  #"""
  def build_model(train_data_to_build): 
    model = keras.Sequential([
      keras.layers.Dense(100, activation=tf.nn.relu,
                         input_shape=(train_data_to_build.shape[1],)),
      keras.layers.Dense(100, activation=tf.nn.relu),
      keras.layers.Dense(100, activation=tf.nn.relu),
      keras.layers.Dense(1)
    ])

    myoptimizer = tf.train.AdamOptimizer(0.0001)

    """
    def myloss(y_true,y_pred):
      
      
      customloss=keras.losses.mean_squared_error(y_true, y_pred)
      #INTRODUCE KUPIEC
      #int_y1 = K.cast([100*x for x in y_pred],'int32')
      
      returntensor = tf.convert_to_tensor(df['d_return'][nh:])
      #int_y2 = K.cast([100*x for x in df['d_return']],'int32')
      #int_y2 = K.cast([x for x in returntensor],'int32')
      
      y_pred = K.cast(y_pred, 'float32')
      returntensor = K.cast(returntensor, 'float32')
     
      c = K.cast(K.greater(returntensor,y_pred), 'float32')
      #c=tf.reduce_sum(errorGreater)
      total = K.cast(K.greater(y_pred,-1000),'float32')
      #total=tf.reduce_sum(errorGreater2)
      
      alpha = 1-cl
      p = c/total
      
      n0 = total-c
      n1 = c

      uc_h0 = n0 * np.log(1 - alpha) + n1 * np.log(alpha)
      uc_h1 = n0 * K.log(1 - p) + n1 * K.log(p)
      uc = -2 * (uc_h0 - uc_h1)
      #chi = tfp.distributions.Chi2(1, validate_args=False, allow_nan_stats=True, name='Chi2')
      chi = tf.distributions.Gamma(concentration=0.5 , rate=0.5, validate_args=False, allow_nan_stats=True, name='Gamma')
      #chi = tf.contrib.distributions.chi2(1)
      prob = 1-chi.cdf(uc, name= 'cdf')
      
      
      #customloss = p  #NO GRADIENTS AVAILABLE
      
      #customloss = keras.losses.mean_squared_error(y_true, y_pred) + p #'SOME' FAIL TO DIFFERENTIATE
      #k=5
      #customloss = k*(1-prob)+customloss
      customloss=customloss+penalty*(1-cl-p)*(1-cl-p)
      return customloss
    """
    
    def myloss(y_true,y_pred):
      
      customloss=keras.losses.mean_squared_error(y_true, y_pred)

      #INTRODUCE KUPIEC
      #int_y1 = K.cast([100*x for x in y_pred],'int32')
      
      returntensor = tf.convert_to_tensor(df['d_return'][nh:])
      #int_y2 = K.cast([100*x for x in df['d_return']],'int32')
      #int_y2 = K.cast([x for x in returntensor],'int32')
      
      y_pred = K.cast(y_pred, 'float32')
      returntensor = K.cast(returntensor, 'float32')
     
      #errorGreater = K.cast(K.greater(y_pred ,returntensor), 'float32')
      errorGreater = K.cast(K.greater(y_pred,returntensor), 'float32')
      c=tf.reduce_sum(errorGreater)
      errorGreater2 = K.cast(K.greater(y_pred,-10000),'float32')
      total=tf.reduce_sum(errorGreater2)
      
      alpha = 1-cl
      p = c/total
      
      n0 = total-c
      n1 = c

      uc_h0 = n0 * np.log(1 - alpha) + n1 * np.log(alpha)
      uc_h1 = n0 * K.log(1 - p) + n1 * K.log(p)
      uc = -2 * (uc_h0 - uc_h1)
      #chi = tfp.distributions.Chi2(1, validate_args=False, allow_nan_stats=True, name='Chi2')
      chi = tf.distributions.Gamma(concentration=0.5 , rate=0.5, validate_args=False, allow_nan_stats=True, name='Gamma')
      #chi = tf.contrib.distributions.chi2(1)
      prob = 1-chi.cdf(uc, name= 'cdf')
      
      
      #customloss = p  #NO GRADIENTS AVAILABLE
      
      #customloss = keras.losses.mean_squared_error(y_true, y_pred) + p #'SOME' FAIL TO DIFFERENTIATE
      #k=5
      #customloss = k*(1-prob)+customloss
      #customloss = k*(1-prob)+customloss
      #customloss=10*(cl-p)*(cl-p)+customloss
      #customloss=penalty*(cl-p)*(cl-p)+customloss
      customloss=penalty*(cl-(1-p))*(cl-(1-p))+customloss
      return customloss
    
    def metric_p(y_true,y_pred):
      
      
      customloss=keras.losses.mean_squared_error(y_true, y_pred)

      #INTRODUCE KUPIEC
      #int_y1 = K.cast([100*x for x in y_pred],'int32')
      
      returntensor = tf.convert_to_tensor(df['d_return'][nh:])
      #int_y2 = K.cast([100*x for x in df['d_return']],'int32')
      #int_y2 = K.cast([x for x in returntensor],'int32')
      
      y_pred = K.cast(y_pred, 'float32')
      returntensor = K.cast(returntensor, 'float32')
     
      c = K.cast(K.greater(returntensor,y_pred), 'float32')
      #c=tf.reduce_sum(errorGreater)
      total = K.cast(K.greater(y_pred,-1000),'float32')
      #total=tf.reduce_sum(errorGreater2)
      
      alpha = 1-cl
      p = c/total
      
      n0 = total-c
      n1 = c

      uc_h0 = n0 * np.log(1 - alpha) + n1 * np.log(alpha)
      uc_h1 = n0 * K.log(1 - p) + n1 * K.log(p)
      uc = -2 * (uc_h0 - uc_h1)
      #chi = tfp.distributions.Chi2(1, validate_args=False, allow_nan_stats=True, name='Chi2')
      chi = tf.distributions.Gamma(concentration=0.5 , rate=0.5, validate_args=False, allow_nan_stats=True, name='Gamma')
      #chi = tf.contrib.distributions.chi2(1)
      prob = 1-chi.cdf(uc, name= 'cdf')
      
      
      #customloss = p  #NO GRADIENTS AVAILABLE
      
      #customloss = keras.losses.mean_squared_error(y_true, y_pred) + p #'SOME' FAIL TO DIFFERENTIATE
      #k=5
      #customloss = k*(1-prob)+customloss
      
      return p
    
    def metric_dev(y_true,y_pred):
      
      
      customloss=keras.losses.mean_squared_error(y_true, y_pred)

      #INTRODUCE KUPIEC
      #int_y1 = K.cast([100*x for x in y_pred],'int32')
      
      returntensor = tf.convert_to_tensor(df['d_return'][nh:])
      #int_y2 = K.cast([100*x for x in df['d_return']],'int32')
      #int_y2 = K.cast([x for x in returntensor],'int32')
      
      y_pred = K.cast(y_pred, 'float32')
      returntensor = K.cast(returntensor, 'float32')
     
      c = K.cast(K.greater(y_pred,returntensor), 'float32')
      #c=tf.reduce_sum(errorGreater)
      total = K.cast(K.greater(y_pred,-10000),'float32')
      #total=tf.reduce_sum(errorGreater2)
      
      alpha = 1-cl
      p = c/total #NO PARSING
      diff = 1-p
      n0 = (total[0,0]-c[0,0])*len(df['d_return'][nh:])
      n1 = c[0,0]*len(df['d_return'][nh:])
      uc_h0 = n0 * np.log(1 - alpha) + n1 * np.log(alpha)
      #unitytf=tf.constant(1.00000, dtype='float32',shape=c[0].get_shape())
      #diff = tf.math.subtract(unitytf,c[0])
      #with sess.as_default():
       # def takelog(x):
        #  return tf.log(x)
      #y=sess.run(diff)
      
      #diff=y[0]
      #logdiff = np.log(y[0])
      
      log1 = 0
      log2 = K.log(c[0])
      
      uc_h1 = n0 * log1 + n1 * log2
      #uc_h1 = n0 * K.log(diff[0]) + n1 * K.log(c[0,0])
      #uc_h1 = n0 * K.log(1 - c[0,0]) + n1 * K.log(c[0,0])
      #uc_h1 =K.log(1-c[0,0])
      #uc_h1 = n0 * np.log(1 - p) + n1 * np.log(p)
      uc = -2 * (uc_h0 - uc_h1)
      #chi = tfp.distributions.Chi2(1, validate_args=False, allow_nan_stats=True, name='Chi2')
      chi = tf.distributions.Gamma(concentration=0.5 , rate=0.5, validate_args=False, allow_nan_stats=True, name='Gamma')
      #chi = tf.contrib.distributions.chi2(1)
      prob = 1-chi.cdf(uc, name= 'cdf')
      
      
      #customloss = p  #NO GRADIENTS AVAILABLE
      
      #customloss = keras.losses.mean_squared_error(y_true, y_pred) + p #'SOME' FAIL TO DIFFERENTIATE
      #k=5
      #customloss = k*(1-prob)+customloss
      
      #return cl-p
      return (1-cl-p)
 
    def metric_total(y_true,y_pred):
      
      
      customloss=keras.losses.mean_squared_error(y_true, y_pred)

      #INTRODUCE KUPIEC
      #int_y1 = K.cast([100*x for x in y_pred],'int32')
      
      returntensor = tf.convert_to_tensor(df['d_return'][nh:])
      #int_y2 = K.cast([100*x for x in df['d_return']],'int32')
      #int_y2 = K.cast([x for x in returntensor],'int32')
      
      y_pred = K.cast(y_pred, 'float32')
      returntensor = K.cast(returntensor, 'float32')
     
      c= K.cast(K.greater(y_pred,returntensor), 'float32')
      #c=tf.reduce_sum(errorGreater)
      total = K.cast(K.greater(y_pred,-10000),'float32')
      #total=tf.reduce_sum(errorGreater2)
      
      alpha = 1-cl
      p = c/total
      
      n0 = total-c
      n1 = c

      uc_h0 = n0 * np.log(1 - alpha) + n1 * np.log(alpha)
      uc_h1 = n0 * K.log(1 - p) + n1 * K.log(p)
      uc = -2 * (uc_h0 - uc_h1)
      #chi = tfp.distributions.Chi2(1, validate_args=False, allow_nan_stats=True, name='Chi2')
      chi = tf.distributions.Gamma(concentration=0.5 , rate=0.5, validate_args=False, allow_nan_stats=True, name='Gamma')
      #chi = tf.contrib.distributions.chi2(1)
      prob = 1-chi.cdf(uc, name= 'cdf')
      
      
      #customloss = p  #NO GRADIENTS AVAILABLE
      
      #customloss = keras.losses.mean_squared_error(y_true, y_pred) + p #'SOME' FAIL TO DIFFERENTIATE
      #k=5
      #customloss = k*(1-prob)+customloss
      
      return total
  
    def metric_c(y_true,y_pred):
      
      
      customloss=keras.losses.mean_squared_error(y_true, y_pred)

      #INTRODUCE KUPIEC
      #int_y1 = K.cast([100*x for x in y_pred],'int32')
      
      returntensor = tf.convert_to_tensor(-df['d_return'][nh:])
      #int_y2 = K.cast([100*x for x in df['d_return']],'int32')
      #int_y2 = K.cast([x for x in returntensor],'int32')
      
      y_pred = K.cast(y_pred, 'float32')
      returntensor = K.cast(returntensor, 'float32')
     
      c = K.cast(K.greater(y_pred,returntensor), 'float32')
      #c=tf.reduce_sum(errorGreater)
      total = K.cast(K.greater(y_pred,-10000),'float32')
      #total=tf.reduce_sum(errorGreater2)
      
      alpha = 1-cl
      p = c/total
      
      n0 = total-c
      n1 = c

      uc_h0 = n0 * np.log(1 - alpha) + n1 * np.log(alpha)
      uc_h1 = n0 * K.log(1 - p) + n1 * K.log(p)
      uc = -2 * (uc_h0 - uc_h1)
      #chi = tfp.distributions.Chi2(1, validate_args=False, allow_nan_stats=True, name='Chi2')
      chi = tf.distributions.Gamma(concentration=0.5 , rate=0.5, validate_args=False, allow_nan_stats=True, name='Gamma')
      #chi = tf.contrib.distributions.chi2(1)
      prob = 1-chi.cdf(uc, name= 'cdf')
      
      
      #customloss = p  #NO GRADIENTS AVAILABLE
      
      #customloss = keras.losses.mean_squared_error(y_true, y_pred) + p #'SOME' FAIL TO DIFFERENTIATE
      #k=5
      #customloss = k*(1-prob)+customloss
      
      return c

    model.compile(loss=myloss, 
                  optimizer=myoptimizer,
                  metrics=[metric_p, metric_dev, metric_total,metric_c,'mae'])
    return model

  model_rough = build_model(train_data)
  model_rough.summary()
  model_wo_aor = build_model(train_data_wo_aor)
  model_wo_aor.summary()

  # Display training progress by printing a single dot for each completed epoch
  class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
      if epoch % 1000 == 0: print('.',end=' ')

  EPOCHS = 100
  print('Progress:       (.=10%)     ')
  # Store training stats
  history_rough = model_rough.fit(train_data, train_labels, epochs=EPOCHS,
                      validation_split=0.2, verbose=0,
                      callbacks=[PrintDot()])
  history_wo_aor = model_wo_aor.fit(train_data_wo_aor, train_labels, epochs=EPOCHS,
                      validation_split=0.2, verbose=0,
                      callbacks=[PrintDot()])
  
  def plot_history(history):
    plt.subplot(111)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.plot(history.epoch, np.array(history.history['loss']),
             label='Train Loss',color='r')
    plt.plot(history.epoch, np.array(history.history['val_loss']),
             label = 'Val loss',color='g')
    plt.legend()
    #plt.ylim([0,0.40])
    plt.show()

  # The patience parameter is the amount of epochs to check for improvement
  #early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
  
  #history = model.fit(train_data, train_labels, epochs=EPOCHS,
   #                   validation_split=0.2, verbose=0,
    #                  callbacks=[early_stopper, PrintDot()])

  plot_history(history_rough)
  plot_history(history_wo_aor)
  
  [loss, metric_p, metric_dev, metric_total,metric_c, mae] = model_rough.evaluate(test_data, test_labels, verbose=1)
  [loss2, metric_p2, metric_dev2, metric_total2,metric_c2, mae2] = model_wo_aor.evaluate(test_data_wo_aor, test_labels, verbose=1)
  

  print("Testing set prob loss: {:7.7f}".format(metric_dev))
  print("Testing set mae loss: {:7.7f}".format(mae))
  
  print("Testing set prob loss (wo roughness): {:7.7f}".format(metric_dev2))
  print("Testing set mae loss (wo roughness): {:7.7f}".format(mae2))
  
  res = pd.DataFrame(data=None, dtype=np.int8)
  res['Return']=None
  res['Return']=df['d_return'][nh:n]
  
  ### Insert Kupiec Test Set
  res_test = pd.DataFrame(data=None, dtype=np.int8)
  res_test['returns']=validation_returns
  res_test['test_predictions']=None
  res_test['test_predictions_wo_aor']=None
  res_test['test_nvar']=None
  res_test['test_gvar']=None
  res_test['test_fvar']=None
  res_test['test_FHS']=None
  res_test['test_logvar']=None
  res_test['test_HS']=None
  
  test_predictions = model_rough.predict(test_data)
  train_predictions = model_rough.predict(train_data)
  test_predictions_wo_aor=model_wo_aor.predict(test_data_wo_aor)
  train_predictions_wo_aor=model_wo_aor.predict(train_data_wo_aor)
  res_test['test_predictions']=test_predictions
  res_test['test_predictions_wo_aor']=test_predictions_wo_aor
  #res_test['returns']=validation_returns
  res_test['test_nvar']=test_nvar
  res_test['test_gvar']=test_gvar
  res_test['test_fvar']=test_fvar
  res_test['test_FHS']=test_FHS
  res_test['test_logvar']=test_logvar
  res_test['test_HS']=test_HS
  
  
  print("Backtest Rough Model out-of-sample "+ticker)
  bt_test_m=Backtest(res_test['test_predictions'],res_test['returns'],(1-cl))
  [kupiec,kupiecp]=bt_test_m.lr_bt()
  #if(kupiecp!=0):
    #bt.plot()
  print("Kupiec-Christoffersen loglikelihood framework")
  print(bt_test_m.lr_bt())
  print("Dynamic Quantile Test (Engle & Manganelli)")
  print(bt_test_m.dq_bt())
  kupiec_c_model_oos=kupiec["p-value"][2]
  kupiec_uc_model_oos=kupiec["p-value"][0]
  kupiecp_model_oos=kupiecp
  #bt_test_m.plot()
  #print("Kupiec-Christoffersen loglikelihood on test set: Model")
  #print(bt_test_m.lr_bt())
  #print("Dynamic Quantile Test (Engle & Manganelli) on test set: Model")
  #print(bt_test_m.dq_bt())
  
  print("Backtest Combination Model out-of-sample "+ticker)
  bt_test_m_wo_aor=Backtest(res_test['test_predictions_wo_aor'],res_test['returns'],(1-cl))
  [kupiec,kupiecp]=bt_test_m_wo_aor.lr_bt()
  #if(kupiecp!=0):
    #bt.plot()
  print("Kupiec-Christoffersen loglikelihood framework")
  print(bt_test_m_wo_aor.lr_bt())
  print("Dynamic Quantile Test (Engle & Manganelli)")
  print(bt_test_m_wo_aor.dq_bt())
  kupiec_c_model_wo_aor_oos=kupiec["p-value"][2]
  kupiec_uc_model_wo_aor_oos=kupiec["p-value"][0]
  kupiecp_model_wo_aor_oos=kupiecp
  #bt_test_m.plot()
  #print("Kupiec-Christoffersen loglikelihood on test set: Model")
  #print(bt_test_m.lr_bt())
  #print("Dynamic Quantile Test (Engle & Manganelli) on test set: Model")
  #print(bt_test_m.dq_bt())
  
  print("Backtest Normal out-of-sample "+ticker)
  bt_test_n=Backtest(res_test['test_nvar'],res_test['returns'],(1-cl))
  #bt_test_n.plot()
  
  [kupiec,kupiecp]=bt_test_n.lr_bt()
  #if(kupiecp!=0):
    #bt_test_n.plot()
  print("Kupiec-Christoffersen loglikelihood framework")
  print(bt_test_n.lr_bt())
  print("Dynamic Quantile Test (Engle & Manganelli)")
  print(bt_test_n.dq_bt())
  kupiec_c_normal_oos=kupiec["p-value"][2]
  kupiec_uc_normal_oos=kupiec["p-value"][0]
  kupiecp_normal_oos=kupiecp
    
  #print("Kupiec-Christoffersen loglikelihood on test set: Normal")
  #print(bt_test_n.lr_bt())
  #print("Dynamic Quantile Test (Engle & Manganelli) on test set: Normal")
  #print(bt_test_n.dq_bt())
  
  
  print("Backtest Gumbel out-of-sample "+ticker)
  bt_test_g=Backtest(res_test['test_gvar'],res_test['returns'],(1-cl))
  #bt_test_g.plot()
  
  [kupiec,kupiecp]=bt_test_g.lr_bt()
  #if(kupiecp!=0):
    #bt_test_g.plot()
  print("Kupiec-Christoffersen loglikelihood framework")
  print(bt_test_g.lr_bt())
  print("Dynamic Quantile Test (Engle & Manganelli)")
  print(bt_test_g.dq_bt())
  kupiec_c_gumbel_oos=kupiec["p-value"][2]
  kupiec_uc_gumbel_oos=kupiec["p-value"][0]
  kupiecp_gumbel_oos=kupiecp
  #print("Kupiec-Christoffersen loglikelihood on test set: Gumbel")
  #print(bt_test_g.lr_bt())
  #print("Dynamic Quantile Test (Engle & Manganelli) on test set: Gumbel")
  #print(bt_test_g.dq_bt())
  
  print("Backtest HS out-of-sample "+ticker)
  bt_test_hs=Backtest(res_test['test_HS'],res_test['returns'],(1-cl))
  [kupiec,kupiecp]=bt_test_hs.lr_bt()
  #if(kupiecp!=0):
    #bt_test_hs.plot()
  print("Kupiec-Christoffersen loglikelihood framework")
  print(bt_test_hs.lr_bt())
  print("Dynamic Quantile Test (Engle & Manganelli)")
  print(bt_test_hs.dq_bt())
  kupiec_c_hs_oos=kupiec["p-value"][2]
  kupiec_uc_hs_oos=kupiec["p-value"][0]
  kupiecp_hs_oos=kupiecp
  #bt_test_g.plot()
  #print("Kupiec-Christoffersen loglikelihood on test set: HS")
  #print(bt_test_hs.lr_bt())
  #print("Dynamic Quantile Test (Engle & Manganelli) on test set: HS")
  #print(bt_test_hs.dq_bt())
  
  print("Backtest lognormal out-of-sample "+ticker)
  bt_test_logvar=Backtest(res_test['test_logvar'],res_test['returns'],(1-cl))
  [kupiec,kupiecp]=bt_test_logvar.lr_bt()
  #if(kupiecp!=0):
    #bt_test_hs.plot()
  print("Kupiec-Christoffersen loglikelihood framework")
  print(bt_test_logvar.lr_bt())
  print("Dynamic Quantile Test (Engle & Manganelli)")
  print(bt_test_logvar.dq_bt())
  kupiec_c_logvar_oos=kupiec["p-value"][2]
  kupiec_uc_logvar_oos=kupiec["p-value"][0]
  kupiecp_logvar_oos=kupiecp
  #bt_test_g.plot()
  #print("Kupiec-Christoffersen loglikelihood on test set: HS")
  #print(bt_test_hs.lr_bt())
  #print("Dynamic Quantile Test (Engle & Manganelli) on test set: HS")
  #print(bt_test_hs.dq_bt())
  
  print("Backtest FHS out-of-sample "+ticker)
  bt_test_fhs=Backtest(res_test['test_FHS'],res_test['returns'],(1-cl))
  [kupiec,kupiecp]=bt_test_fhs.lr_bt()
  #if(kupiecp!=0):
    #bt_test_hs.plot()
  print("Kupiec-Christoffersen loglikelihood framework")
  print(bt_test_fhs.lr_bt())
  print("Dynamic Quantile Test (Engle & Manganelli)")
  print(bt_test_fhs.dq_bt())
  kupiec_c_fhs_oos=kupiec["p-value"][2]
  kupiec_uc_fhs_oos=kupiec["p-value"][0]
  kupiecp_fhs_oos=kupiecp
  #bt_test_g.plot()
  #print("Kupiec-Christoffersen loglikelihood on test set: HS")
  #print(bt_test_hs.lr_bt())
  #print("Dynamic Quantile Test (Engle & Manganelli) on test set: HS")
  #print(bt_test_hs.dq_bt())
  
  print("Backtest FVaR out-of-sample "+ticker)
  bt_test_fvar=Backtest(res_test['test_fvar'],res_test['returns'],(1-cl))
  [kupiec,kupiecp]=bt_test_fvar.lr_bt()
  #if(kupiecp!=0):
    #bt_test_hs.plot()
  print("Kupiec-Christoffersen loglikelihood framework")
  print(bt_test_fvar.lr_bt())
  print("Dynamic Quantile Test (Engle & Manganelli)")
  print(bt_test_fvar.dq_bt())
  kupiec_c_fvar_oos=kupiec["p-value"][2]
  kupiec_uc_fvar_oos=kupiec["p-value"][0]
  kupiecp_fvar_oos=kupiecp
  #bt_test_g.plot()
  #print("Kupiec-Christoffersen loglikelihood on test set: HS")
  #print(bt_test_hs.lr_bt())
  #print("Dynamic Quantile Test (Engle & Manganelli) on test set: HS")
  #print(bt_test_hs.dq_bt())
   
  
  #predictions = model.predict(train_data)
  
  #"""
  #####
  
  ### GENETIC ALGOS ### 
  ### ON / OFF ###
  """
  generations = nr_of_generations  # Number of times to evole the population.
  population = population_size  # Number of networks in each generation.
  dataset = 'thesis'

  nn_param_choices = {
        'nb_neurons': [2, 4, 8, 16, 32, 64, 128, 256, 512],
        'nb_layers': [1, 2, 3, 4],
        'activation': ['relu', 'elu', 'tanh', 'sigmoid'],
        'optimizer': ['rmsprop', 'adam', 'sgd', 'adagrad',
                      'adadelta', 'adamax', 'nadam'],
    }

  print("***Evolving %d generations with population %d***" %
                 (generations, population))
  print('                                                   ')
  print('                                                   ')
  
  train_data=training_examples 
  train_labels=training_targets
  test_data=validation_examples
  test_labels=validation_targets
  result = generate(generations, population, nn_param_choices, dataset, train_data ,test_data,train_labels,test_labels,df,nh,n)
  #include predictions
  
  print_networks(result)
  test_predictions = get_network_test_predictions(result)
  train_predictions = get_network_train_predictions(result)
  
  res = pd.DataFrame(data=None, dtype=np.int8)
  res['Return']=None
  res['Return']=df['d_return'][nh:n]
  
  ### Insert Kupiec Test Set
  res_test = pd.DataFrame(data=None, dtype=np.int8)
  res_test['returns']=validation_returns
  res_test['test_predictions']=None
  res_test['test_nvar']=None
  res_test['test_gvar']=None
  res_test['test_HS']=None
  
  #test_predictions = model.predict(test_data)
  res_test['test_predictions']=test_predictions
  #res_test['returns']=validation_returns
  res_test['test_nvar']=test_nvar
  res_test['test_gvar']=test_gvar
  res_test['test_HS']=test_HS
  bt_test_m=Backtest(res_test['test_predictions'],res_test['returns'],(1-cl))
  #bt_test_m.plot()
  print("Kupiec-Christoffersen loglikelihood on test set: Model")
  print(bt_test_m.lr_bt())
  print("Dynamic Quantile Test (Engle & Manganelli) on test set: Model")
  print(bt_test_m.dq_bt())
  bt_test_n=Backtest(res_test['test_nvar'],res_test['returns'],(1-cl))
  #bt_test_n.plot()
  print("Kupiec-Christoffersen loglikelihood on test set: Normal")
  print(bt_test_n.lr_bt())
  print("Dynamic Quantile Test (Engle & Manganelli) on test set: Normal")
  print(bt_test_n.dq_bt())
  bt_test_g=Backtest(res_test['test_gvar'],res_test['returns'],(1-cl))
  #bt_test_g.plot()
  print("Kupiec-Christoffersen loglikelihood on test set: Gumbel")
  print(bt_test_g.lr_bt())
  print("Dynamic Quantile Test (Engle & Manganelli) on test set: Gumbel")
  print(bt_test_g.dq_bt())
  bt_test_g=Backtest(res_test['test_HS'],res_test['returns'],(1-cl))
  #bt_test_g.plot()
  print("Kupiec-Christoffersen loglikelihood on test set: HS")
  print(bt_test_g.lr_bt())
  print("Dynamic Quantile Test (Engle & Manganelli) on test set: HS")
  print(bt_test_g.dq_bt())
 
  """
  ########
  
  res['Prediction']=None
  #res['Prediction']=-train_predictions
  res['Prediction']=-train_predictions
  plt.subplot(111)
  plt.plot(res['Prediction'],c='b')
  plt.plot(-res['Prediction'],c='b')
  plt.plot(res['Return'],c='g')
  _=plt.legend()
  plt.show()
  
  plt.subplot(111)
  plt.plot(df['NVaR'][nh:n], label = 'NVaR',c='c')
  plt.plot(-df['NVaR'][nh:n],c='c')
  plt.plot(df['d_return'][nh:n], label = 'Return',c='g')
  _=plt.legend()
  plt.show()

  plt.subplot(111)
  plt.plot(df['TVaR'][nh:n], label = 'TVaR',c='red')
  plt.plot(-df['TVaR'][nh:n],c='red')
  plt.plot(df['d_return'][nh:n], label = 'Return',c='g')
  _=plt.legend()
  plt.show()

  plt.subplot(111)
  plt.plot(df['logVaR'][nh:n], label = 'logVaR',c='y')
  plt.plot(-df['logVaR'][nh:n],c='y')
  plt.plot(df['d_return'][nh:n], label = 'Return',c='g')
  _=plt.legend()
  plt.show()
 
  plt.subplot(111)
  plt.plot(df['FVaR'][nh:n], label = 'FVaR',c='y')
  plt.plot(-df['FVaR'][nh:n],c='y')
  plt.plot(df['d_return'][nh:n], label = 'Return',c='g')
  _=plt.legend()
  plt.show()
 
  plt.subplot(111)
  plt.plot(df['GVaR'][nh:n], label = 'GVaR',c='y')
  plt.plot(-df['GVaR'][nh:n],c='y')
  plt.plot(df['d_return'][nh:n], label = 'Return',c='g')
  _=plt.legend()
  plt.show()
  
  plt.subplot(111)
  plt.plot(df['HS'][nh:n], label = 'HS',c='y')
  plt.plot(-df['HS'][nh:n],c='y')
  plt.plot(df['d_return'][nh:n], label = 'Return',c='g')
  _=plt.legend()
  plt.show()
  
  plt.subplot(111)
  plt.plot(df['FHS'][nh:n], label = 'FHS',c='y')
  plt.plot(-df['FHS'][nh:n],c='y')
  plt.plot(df['d_return'][nh:n], label = 'Return',c='g')
  _=plt.legend()
  plt.show()
 
 
 
  exceptionsn=0
  for i in range(nh,n):
    if (df['NVaR'][i]<-df['d_return'][i+1]):
      exceptionsn = exceptionsn+1

  exceptionst=0
  for i in range(nh,n):
    if (df['TVaR'][i]<-df['d_return'][i+1]):
      exceptionst = exceptionst+1

  exceptionsl=0
  for i in range(nh,n):
    if (df['logVaR'][i]<-df['d_return'][i+1]):
      exceptionsl = exceptionsl+1


  #exceptionsm=0
  #for i in range(nh,nh+len(res['Prediction'])-1):
   # if (res['Prediction'][i]>res['Return'][i+1]):
    #  exceptionsm = exceptionsm+1

  #print("Exceptions Model "+str(exceptionsm))
  #print("Exceptions NVaR "+str(exceptionsn))
  #print("Exceptions TVaR "+str(exceptionst))
  #print("Exceptions logVaR "+str(exceptionsl))

  bt=Backtest(res['Return'],res['Prediction'],(1-cl))
  [kupiec,kupiecp_model_is]=bt.lr_bt()
  #if(kupiecp_model_is!=0):
    #bt.plot()
  print("Kupiec-Christoffersen loglikelihood framework")
  print(bt.lr_bt())
  print("Dynamic Quantile Test (Engle & Manganelli)")
  print(bt.dq_bt())
  kupiec_c_model_is=kupiec["p-value"][2]
  kupiec_uc_model_is=kupiec["p-value"][0]
  kupiecp_model_is=kupiecp_model_is
  
  
  res['Prediction_wo_aor']=None
  res['Prediction_wo_aor']=-train_predictions_wo_aor
  bt_wo_aor=Backtest(res['Return'],res['Prediction_wo_aor'],(1-cl))
  [kupiec,kupiecp_model_is]=bt_wo_aor.lr_bt()
  #if(kupiecp_model_is!=0):
    #bt.plot()
  print("Kupiec-Christoffersen loglikelihood framework")
  print(bt_wo_aor.lr_bt())
  print("Dynamic Quantile Test (Engle & Manganelli)")
  print(bt_wo_aor.dq_bt())
  kupiec_c_model_wo_aor_is=kupiec["p-value"][2]
  kupiec_uc_model_wo_aor_is=kupiec["p-value"][0]
  kupiecp_model_wo_aor_is=kupiecp_model_is
  
  return res,nh,df,kupiec_c_model_is,kupiec_uc_model_is,kupiecp_model_is,kupiec_c_model_wo_aor_is,kupiec_uc_model_wo_aor_is,kupiecp_model_wo_aor_is,kupiec_c_model_oos,kupiec_uc_model_oos,kupiecp_model_oos,kupiec_c_model_wo_aor_oos,kupiec_uc_model_wo_aor_oos,kupiecp_model_wo_aor_oos,kupiec_c_normal_oos,kupiec_uc_normal_oos,kupiecp_normal_oos,kupiec_c_gumbel_oos,kupiec_uc_gumbel_oos,kupiecp_gumbel_oos,kupiec_c_hs_oos,kupiec_uc_hs_oos,kupiecp_hs_oos,kupiec_c_fhs_oos,kupiec_uc_fhs_oos,kupiecp_fhs_oos,kupiec_c_fvar_oos,kupiec_uc_fvar_oos,kupiecp_fvar_oos,kupiec_c_logvar_oos,kupiec_uc_logvar_oos,kupiecp_logvar_oos
  

################################################################################
##### MAIN CODE: EXTRACTION OF DATA AND SUMMARY DATAFRAME                  #####
################################################################################

from google.colab import files
penalty=0 #rmse loss
count=0


nr_of_generations = 10 #@param {type:"integer"}
population_size = 10 #@param {type:"integer"}
startticker = 20 #@param {type:"integer"}
count=startticker
endticker = 50 #@param {type:"integer"}
penalty = 10 #@param {type:"integer"}
eikon_user_id ="" #@param {type:"string"}
eikon_password ="" #@param {type:"string"}



DWE=Datastream(username=eikon_user_id, password=eikon_password)

countries = ['BG','CN','FR','BD','IT','JP','NL','SD','SW','UK','US']
industries = ['OILGS','BMATR','INDUS','CNSMG','HLTHC','CNSMS','TELCM','UTILS','FINAN','TECNO']
#countries = ['AU','CN','FR','BD','IT','JP','NL','ES','SD','SW','UK','US'] #ZONDER BG
#industries = ['OILGS','BMATR','INDUS','HLTHC''CNSMG','CNSMS','TELCM','UTILS','FINAN','TECNO'] 
#countries = ['BG']
#industries = ['OILGS']

namelist =[]
countrylist=[]
industrylist=[]
for c in countries:
  for i in industries:
    res = DWE.get_constituents(i+c)
    for a in res['IBTKR'][:10]:
      namelist.append(a)
      countrylist.append(c)
      industrylist.append(i)
crosssectionaldata = pd.DataFrame(data={'Ticker':namelist,'Country':countrylist,'Industry':industrylist})

tickerlist=crosssectionaldata['Ticker']
countrylists=crosssectionaldata['Country']
industrylists=crosssectionaldata['Industry']


summ = pd.DataFrame(data=None, dtype=None)
summ['Ticker']=tickerlist
summ['Country']=countrylists
summ['Industry']=industrylists
summ['Average 10-day MVaR']=None
summ['Average 10-day Return']=None
summ['Performance ratio']=None
summ['Average Higushi fractal dimension']=None
summ['Average Katz fractal dimension']=None
summ['Average Hurst Exponent']=None
summ['UR rough model IS']=None
summ['CR rough model IS']=None
summ['Violation ratio rough model IS']=None
summ['UR combination model IS']=None
summ['CR combination model IS']=None
summ['Violation ratio combination model IS']=None
summ['UR rough model OOS']=None
summ['CR rough model OOS']=None
summ['Violation ratio rough model OOS']=None
summ['UR combination model OOS']=None
summ['CR combination model OOS']=None
summ['Violation ratio combination model OOS']=None
summ['UR normal model OOS']=None
summ['CR normal model OOS']=None
summ['Violation ratio normal model OOS']=None
summ['UR Gumbel model OOS']=None
summ['CR Gumbel model OOS']=None
summ['Violation ratio Gumbel model OOS']=None
summ['UR HS model OOS']=None
summ['CR HS model OOS']=None
summ['Violation ratio HS model OOS']=None
summ['UR FHS model OOS']=None
summ['CR FHS model OOS']=None
summ['Violation ratio FHS model OOS']=None
summ['UR Fréchet model OOS']=None
summ['CR Fréchet model OOS']=None
summ['Violation ratio Fréchet model OOS']=None
summ['UR lognormal model OOS']=None
summ['CR lognormal model OOS']=None
summ['Violation ratio lognormal model OOS']=None

#summ['Average Shannon Entropy']=None
#result=None
for i in tickerlist[startticker:endticker]:
#for i in tickerlist:
  print("Iteration: "+str(count))
  if(i!='NA'):
    res,nh,df,kupiec_c_model_is,kupiec_uc_model_is,kupiecp_model_is,kupiec_c_model_wo_aor_is,kupiec_uc_model_wo_aor_is,kupiecp_model_wo_aor_is,kupiec_c_model_oos,kupiec_uc_model_oos,kupiecp_model_oos,kupiec_c_model_wo_aor_oos,kupiec_uc_model_wo_aor_oos,kupiecp_model_wo_aor_oos,kupiec_c_normal_oos,kupiec_uc_normal_oos,kupiecp_normal_oos,kupiec_c_gumbel_oos,kupiec_uc_gumbel_oos,kupiecp_gumbel_oos,kupiec_c_hs_oos,kupiec_uc_hs_oos,kupiecp_hs_oos,kupiec_c_fhs_oos,kupiec_uc_fhs_oos,kupiecp_fhs_oos,kupiec_c_fvar_oos,kupiec_uc_fvar_oos,kupiecp_fvar_oos,kupiec_c_logvar_oos,kupiec_uc_logvar_oos,kupiecp_logvar_oos = do(i,count, nr_of_generations, population_size)
    summ['Ticker'][count]=tickerlist[count]
    summ['Average 10-day MVaR'][count]=(-np.mean(res['Prediction'])*100*np.sqrt(10))
    summ['Average 10-day Return'][count]=(np.mean(res['Return'])*100*np.sqrt(10))
    summ['Performance ratio'][count]=(np.mean(res['Return']))/(-np.mean(res['Prediction']))*100
    summ['Average Higushi fractal dimension'][count]=np.mean(df["Higushi_FD_"+str(nh)][nh:])
    summ['Average Katz fractal dimension'][count]=np.mean(df["Katz_"+str(nh)][nh:])
    summ['Average Hurst Exponent'][count]=np.mean(df["Hurst_"+str(nh)][nh:])
    summ['UR rough model IS'][count]=kupiec_uc_model_is
    summ['CR rough model IS'][count]=kupiec_c_model_is
    summ['Violation ratio rough model IS'][count]=kupiecp_model_is
    summ['UR combination model IS'][count]=kupiec_uc_model_wo_aor_is
    summ['CR combination model IS'][count]=kupiec_c_model_wo_aor_is
    summ['Violation ratio combination model IS'][count]=kupiecp_model_wo_aor_is
    summ['UR rough model OOS'][count]=kupiec_uc_model_oos
    summ['CR rough model OOS'][count]=kupiec_c_model_oos
    summ['Violation ratio rough model OOS'][count]=kupiecp_model_oos
    summ['UR combination model OOS'][count]=kupiec_uc_model_wo_aor_oos
    summ['CR combination model OOS'][count]=kupiec_c_model_wo_aor_oos
    summ['Violation ratio combination model OOS'][count]=kupiecp_model_wo_aor_oos
    summ['UR normal model OOS'][count]=kupiec_uc_normal_oos
    summ['CR normal model OOS'][count]=kupiec_c_normal_oos
    summ['Violation ratio normal model OOS'][count]=kupiecp_normal_oos
    summ['UR Gumbel model OOS'][count]=kupiec_uc_gumbel_oos
    summ['CR Gumbel model OOS'][count]=kupiec_c_gumbel_oos
    summ['Violation ratio Gumbel model OOS'][count]=kupiecp_gumbel_oos
    summ['UR HS model OOS'][count]=kupiec_uc_hs_oos
    summ['CR HS model OOS'][count]=kupiec_c_hs_oos
    summ['Violation ratio HS model OOS'][count]=kupiecp_hs_oos
    summ['UR FHS model OOS'][count]=kupiec_uc_fhs_oos
    summ['CR FHS model OOS'][count]=kupiec_c_fhs_oos
    summ['Violation ratio FHS model OOS'][count]=kupiecp_fhs_oos
    summ['UR Fréchet model OOS'][count]=kupiec_uc_fvar_oos
    summ['CR Fréchet model OOS'][count]=kupiec_c_fvar_oos
    summ['Violation ratio Fréchet model OOS'][count]=kupiecp_fvar_oos
    summ['UR lognormal model OOS'][count]=kupiec_uc_logvar_oos
    summ['CR lognormal model OOS'][count]=kupiec_c_logvar_oos
    summ['Violation ratio lognormal model OOS'][count]=kupiecp_logvar_oos
    #summ['Average Shannon Entropy'][count]=np.mean(df["Shannon_"+str(nh)][nh:])

  count=count+1

display.display(summ)
#summ.to_csv("summary.csv")

df=summ[startticker:endticker]
for i in range(startticker,endticker):
#for i in range(startticker,7):
#for i in range(0,len(df['Country'])-1):
  if df['Country'][i]=='JP':
    df['Country'][i]='Japan'
  if df['Country'][i] =='NL':
    df['Country'][i]='The Netherlands'
  if df['Country'][i] =='SD':
    df['Country'][i]='Sweden'
  if df['Country'][i] =='SW':
    df['Country'][i]='Switzerland'
  if df['Country'][i] =='UK':
    df['Country'][i]='UK'
  if df['Country'][i]=='US':
    df['Country'][i] = 'US'
  if df['Country'][i]=='BG':
    df['Country'][i] = 'Belgium'
  if df['Country'][i]=='CN':
    df['Country'][i] = 'Canada'
  if df['Country'][i]=='FR':
    df['Country'][i] = 'France'
  if df['Country'][i]=='BD':
    df['Country'][i] = 'Germany'
  if df['Country'][i]=='IT':
    df['Country'][i] = 'Italy'
    
file_name='result'+str(count)+'.csv'
df.to_csv(file_name)
from google.colab import files
files.download(file_name)
display.display(df)


As you can see, the code uses some parameters that can be adjusted by means of a ** form**.

The **source** is by default datastream ('Refinitiv Eikon API'). One needs to fill in an eikon_user_id (e.g. DS:ABCD007) and a eikon_password (e.g. DUMBO123) in order for it to function. Alternatively, there are some other APIs included, but these are less stable and the code needs some minor adjustments for it to function.

The **starting** and **ending date** speak for themselves and the user is of course free to play with these.

The **confidence level** is the threshold for the market risk measure (i.e. a 99% cl means that the model will 'underpredict' a financial loss with a 1% probability).
The **hurst_n** and **lookback_period** are two user defined time windows that are used for the calibration of the Hurst exponent and standard models respectively. (Note that these parameters are explained in the main text)

The **nr_of_generations** and **population_size** are the two parameters needed for our genetic algorithms (see main text). These algorithms are now 'turned off' (i.e. put in comments), since they are very time-consuming and if only for development or demo purposes.

The **start** and **endticker** is the index of a ticker within our stock universe (ranging from 0 to 870). These parameters allow us to train the models for only a range of tickers instead of all 870 consecutively.

Lastly, the **penalty** variable is a tunable parameter in our custom loss function. It determines how much weigth is given to the exception loss (violation ratio defers from the significance level) versus standard L2 loss (predicted VaR compared to hypothetical real VaR).

